In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Change this if your folder name is different
BASE_DIR = "/content/drive/MyDrive/SkinDiseaseProject"

ZIP_PATH = f"{BASE_DIR}/skin_cleaned_dataset.zip"   # train/val/test inside this
MODEL_DIR   = f"{BASE_DIR}/models"
EXTRACT_DIR = f"{BASE_DIR}/clean_data"

print("Model path:", MODEL_DIR)
print("ZIP:", ZIP_PATH)
print("EXTRACT TO:", EXTRACT_DIR)


Model path: /content/drive/MyDrive/SkinDiseaseProject/models
ZIP: /content/drive/MyDrive/SkinDiseaseProject/skin_cleaned_dataset.zip
EXTRACT TO: /content/drive/MyDrive/SkinDiseaseProject/clean_data


In [ ]:
import os

print("ZIP exists:", os.path.exists(ZIP_PATH))


ZIP exists: True


In [ ]:
import zipfile

if not os.path.exists(EXTRACT_DIR):
    print("Extracting ZIP to Drive...")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_DIR)
    print("Extraction completed.")
else:
    print("Dataset already extracted. Skipping.")


Dataset already extracted. Skipping.


In [ ]:
import os

EXTRACT_DIR = "/content/drive/MyDrive/SkinDiseaseProject/clean_data"

for split in ["train", "val", "test"]:
    path = os.path.join(EXTRACT_DIR, split)
    print(f"\n{split.upper()} exists:", os.path.exists(path))
    if os.path.exists(path):
        print("Classes:", sorted(os.listdir(path)))



TRAIN exists: True
Classes: ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'scc', 'vasc']

VAL exists: True
Classes: ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'scc', 'vasc']

TEST exists: True
Classes: ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'scc', 'vasc']


In [ ]:
# All variants mapped to ONE canonical name
CLASS_MAP = {
    "AK": "akiec",
    "akiec": "akiec",

    "BCC": "bcc",
    "bcc": "bcc",

    "BKL": "bkl",
    "bkl": "bkl",

    "DF": "df",
    "df": "df",

    "MEL": "mel",
    "mel": "mel",

    "NV": "nv",
    "nv": "nv",

    "SCC": "scc",

    "VASC": "vasc",
    "vasc": "vasc",
}


In [ ]:
import os
import shutil

BASE_DIRR = "/content/drive/MyDrive/SkinDiseaseProject/clean_data"


In [ ]:
def merge_class_folders(split_path):
    folders = os.listdir(split_path)

    for folder in folders:
        src = os.path.join(split_path, folder)

        if not os.path.isdir(src):
            continue

        if folder not in CLASS_MAP:
            print("⚠️ Unknown folder skipped:", folder)
            continue

        target_name = CLASS_MAP[folder]
        dst = os.path.join(split_path, target_name)

        # Case 1: already canonical
        if folder == target_name:
            continue

        # Case 2: target folder doesn't exist → rename
        if not os.path.exists(dst):
            os.rename(src, dst)
            print(f"Renamed {folder} → {target_name}")

        # Case 3: target exists → merge images
        else:
            for file in os.listdir(src):
                src_file = os.path.join(src, file)
                dst_file = os.path.join(dst, file)

                # Avoid overwrite
                if os.path.exists(dst_file):
                    name, ext = os.path.splitext(file)
                    dst_file = os.path.join(dst, f"{name}_dup{ext}")

                shutil.move(src_file, dst_file)

            os.rmdir(src)
            print(f"Merged {folder} → {target_name}")


In [ ]:
for split in ["train", "val", "test"]:
    split_path = os.path.join(BASE_DIRR, split)
    print(f"\n🔹 Processing {split.upper()}...")
    merge_class_folders(split_path)



🔹 Processing TRAIN...
⚠️ Unknown folder skipped: scc

🔹 Processing VAL...
⚠️ Unknown folder skipped: scc

🔹 Processing TEST...
⚠️ Unknown folder skipped: scc


In [ ]:
for split in ["train", "val", "test"]:
    path = os.path.join(BASE_DIRR, split)
    print(f"\n{split.upper()} classes:")
    print(sorted(os.listdir(path)))



TRAIN classes:
['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'scc', 'vasc']

VAL classes:
['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'scc', 'vasc']

TEST classes:
['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'scc', 'vasc']


In [ ]:
for split in ["train", "val", "test"]:
    path = os.path.join(EXTRACT_DIR, split)
    print(split, "exists:", os.path.exists(path))
    if os.path.exists(path):
        print("Classes:", sorted(os.listdir(path)))


train exists: True
Classes: ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'scc', 'vasc']
val exists: True
Classes: ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'scc', 'vasc']
test exists: True
Classes: ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'scc', 'vasc']


In [ ]:
import tensorflow as tf

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    f"{EXTRACT_DIR}/train",
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical",
    shuffle=True
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    f"{EXTRACT_DIR}/val",
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical",
    shuffle=False
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    f"{EXTRACT_DIR}/test",
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical",
    shuffle=False
)

class_names = train_ds.class_names
NUM_CLASSES = len(class_names)

print("Classes:", class_names)
print("Total classes:", NUM_CLASSES)


Found 10580 files belonging to 8 classes.
Found 2267 files belonging to 8 classes.
Found 2268 files belonging to 8 classes.
Classes: ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'scc', 'vasc']
Total classes: 8


Train

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras import layers

IMAGE_SIZE = (224,224)
NUM_CLASSES = len(class_names)

base_model = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=IMAGE_SIZE + (3,)
)

base_model.trainable = False

inputs = tf.keras.Input(shape=IMAGE_SIZE + (3,))
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
    run_eagerly=True   # 🔥 IMPORTANT
)


Callback

In [ ]:
CHECKPOINT_PATH = f"{BASE_DIR}/models/checkpoints/best_model.h5"

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        CHECKPOINT_PATH,
        monitor="val_accuracy",
        save_best_only=True,
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=4,
        restore_best_weights=True
    )
]


Training

In [ ]:
EPOCHS = 15

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)


Epoch 1/15
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.4625 - loss: 1.7995 

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf

train_ds = train_ds.cache().prefetch(tf.data.AUTOTUNE)
val_ds   = val_ds.cache().prefetch(tf.data.AUTOTUNE)

print("✅ Dataset cached & prefetched")


✅ Dataset cached & prefetched


In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    initial_epoch=1,   # resume AFTER epoch 1
    callbacks=callbacks
)


Epoch 2/15
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 940ms/step - accuracy: 0.6313 - loss: 1.0753
Epoch 2: val_accuracy improved from -inf to 0.68990, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best_model.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 545s 2s/step - accuracy: 0.6314 - loss: 1.0752 - val_accuracy: 0.6899 - val_loss: 0.8408
Epoch 3/15
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 692ms/step - accuracy: 0.6529 - loss: 0.9759
Epoch 3: val_accuracy improved from 0.68990 to 0.69299, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best_model.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 252s 760ms/step - accuracy: 0.6529 - loss: 0.9758 - val_accuracy: 0.6930 - val_loss: 0.8184
Epoch 4/15
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 705ms/step - accuracy: 0.6692 - loss: 0.9238
Epoch 4: val_accuracy improved from 0.69299 to 0.69916, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best_model.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 287s 837ms/step - accuracy: 0.6692 - loss: 0.9238 - val_accuracy: 0.6992 - val_loss: 0.8080
Epoch 5/15
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 706ms/step - accuracy: 0.6765 - loss: 0.8915
Epoch 5: val_accuracy improved from 0.69916 to 0.70049, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best_model.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 259s 780ms/step - accuracy: 0.6765 - loss: 0.8914 - val_accuracy: 0.7005 - val_loss: 0.8006
Epoch 6/15
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 704ms/step - accuracy: 0.6731 - loss: 0.8817
Epoch 6: val_accuracy improved from 0.70049 to 0.70975, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best_model.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 255s 769ms/step - accuracy: 0.6731 - loss: 0.8816 - val_accuracy: 0.7097 - val_loss: 0.8004
Epoch 7/15
134/331 ━━━━━━━━━━━━━━━━━━━━ 2:17 700ms/step - accuracy: 0.6818 - loss: 0.8888

KeyboardInterrupt: 

Fine-tuining

In [ ]:
# Unfreeze the backbone
base_model.trainable = True

# Freeze first 70% of layers (VERY IMPORTANT)
fine_tune_at = int(len(base_model.layers) * 0.7)

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

print("Total layers:", len(base_model.layers))
print("Trainable layers:", len([l for l in base_model.layers if l.trainable]))


Total layers: 238
Trainable layers: 72


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    initial_epoch=6,   # resume AFTER stage-1
    callbacks=callbacks
)


Epoch 7/20
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.6032 - loss: 1.1952
Epoch 7: val_accuracy did not improve from 0.70975
331/331 ━━━━━━━━━━━━━━━━━━━━ 106s 180ms/step - accuracy: 0.6032 - loss: 1.1950 - val_accuracy: 0.6798 - val_loss: 0.9356
Epoch 8/20
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.6403 - loss: 1.0349
Epoch 8: val_accuracy did not improve from 0.70975
331/331 ━━━━━━━━━━━━━━━━━━━━ 18s 55ms/step - accuracy: 0.6404 - loss: 1.0348 - val_accuracy: 0.6934 - val_loss: 0.8885
Epoch 9/20
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.6587 - loss: 0.9393
Epoch 9: val_accuracy did not improve from 0.70975
331/331 ━━━━━━━━━━━━━━━━━━━━ 18s 55ms/step - accuracy: 0.6588 - loss: 0.9393 - val_accuracy: 0.7005 - val_loss: 0.8523
Epoch 10/20
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.6788 - loss: 0.9016
Epoch 10: val_accuracy improved from 0.70975 to 0.71416, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best

331/331 ━━━━━━━━━━━━━━━━━━━━ 21s 62ms/step - accuracy: 0.6788 - loss: 0.9016 - val_accuracy: 0.7142 - val_loss: 0.8258
Epoch 11/20
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.6882 - loss: 0.8592
Epoch 11: val_accuracy improved from 0.71416 to 0.72651, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best_model.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 20s 61ms/step - accuracy: 0.6882 - loss: 0.8591 - val_accuracy: 0.7265 - val_loss: 0.8070
Epoch 12/20
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.7045 - loss: 0.8164
Epoch 12: val_accuracy improved from 0.72651 to 0.72739, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best_model.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 20s 61ms/step - accuracy: 0.7046 - loss: 0.8164 - val_accuracy: 0.7274 - val_loss: 0.7893
Epoch 13/20
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.7145 - loss: 0.7855
Epoch 13: val_accuracy improved from 0.72739 to 0.72916, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best_model.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 21s 62ms/step - accuracy: 0.7146 - loss: 0.7854 - val_accuracy: 0.7292 - val_loss: 0.7755
Epoch 14/20
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.7255 - loss: 0.7269
Epoch 14: val_accuracy improved from 0.72916 to 0.73180, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best_model.h5


...

331/331 ━━━━━━━━━━━━━━━━━━━━ 21s 63ms/step - accuracy: 0.7411 - loss: 0.7040 - val_accuracy: 0.7367 - val_loss: 0.7537
Epoch 16/20
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.7449 - loss: 0.6925
Epoch 16: val_accuracy did not improve from 0.73666
331/331 ━━━━━━━━━━━━━━━━━━━━ 18s 55ms/step - accuracy: 0.7449 - loss: 0.6924 - val_accuracy: 0.7367 - val_loss: 0.7447
Epoch 17/20
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.7516 - loss: 0.6629
Epoch 17: val_accuracy improved from 0.73666 to 0.73974, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best_model.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 20s 61ms/step - accuracy: 0.7516 - loss: 0.6629 - val_accuracy: 0.7397 - val_loss: 0.7362
Epoch 18/20
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.7640 - loss: 0.6385
Epoch 18: val_accuracy improved from 0.73974 to 0.74548, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best_model.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 21s 62ms/step - accuracy: 0.7640 - loss: 0.6384 - val_accuracy: 0.7455 - val_loss: 0.7294
Epoch 19/20
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.7694 - loss: 0.6226
Epoch 19: val_accuracy improved from 0.74548 to 0.74857, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best_model.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 21s 64ms/step - accuracy: 0.7695 - loss: 0.6226 - val_accuracy: 0.7486 - val_loss: 0.7225
Epoch 20/20
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.7764 - loss: 0.5859
Epoch 20: val_accuracy improved from 0.74857 to 0.75121, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best_model.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 20s 61ms/step - accuracy: 0.7764 - loss: 0.5859 - val_accuracy: 0.7512 - val_loss: 0.7169


In [ ]:
import numpy as np

y_labels = []

for _, y in train_ds.unbatch():
    y_labels.append(np.argmax(y.numpy()))

y_labels = np.array(y_labels)

print("Total samples:", len(y_labels))
print("Unique classes:", np.unique(y_labels))


Total samples: 10580
Unique classes: [0 1 2 3 4 5 6 7]


In [ ]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_labels),
    y=y_labels
)

class_weights = dict(zip(np.unique(y_labels), class_weights))

print("Class weights:")
for k, v in class_weights.items():
    print(f"Class {k}: {v:.3f}")


Class weights:
Class 0: 3.844
Class 1: 1.668
Class 2: 1.030
Class 3: 14.375
Class 4: 0.649
Class 5: 0.228
Class 6: 10.413
Class 7: 13.094


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
history_weighted = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25,
    initial_epoch=17,
    class_weight=class_weights,
    callbacks=callbacks
)


Epoch 18/25
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.7849 - loss: 0.9941
Epoch 18: val_accuracy did not improve from 0.75121
331/331 ━━━━━━━━━━━━━━━━━━━━ 110s 199ms/step - accuracy: 0.7849 - loss: 0.9939 - val_accuracy: 0.7442 - val_loss: 0.7320
Epoch 19/25
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.7774 - loss: 0.7688
Epoch 19: val_accuracy did not improve from 0.75121
331/331 ━━━━━━━━━━━━━━━━━━━━ 18s 55ms/step - accuracy: 0.7774 - loss: 0.7687 - val_accuracy: 0.7375 - val_loss: 0.7514
Epoch 20/25
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.7886 - loss: 0.6750
Epoch 20: val_accuracy did not improve from 0.75121
331/331 ━━━━━━━━━━━━━━━━━━━━ 18s 55ms/step - accuracy: 0.7886 - loss: 0.6749 - val_accuracy: 0.7349 - val_loss: 0.7609
Epoch 21/25
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.7868 - loss: 0.6321
Epoch 21: val_accuracy did not improve from 0.75121
331/331 ━━━━━━━━━━━━━━━━━━━━ 19s 57ms/step - accuracy: 0.7868 - loss: 0.6321 - v

EfficientNetB3 Training

In [ ]:
B3_CHECKPOINT_PATH = "/content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best_model_B3.h5"


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB3


In [ ]:
NUM_CLASSES = 8   # must match your dataset

base_model = EfficientNetB3(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)

base_model.trainable = False  # IMPORTANT (Stage-1)

inputs = layers.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = models.Model(inputs, outputs)
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb3 (Functional)     │ (None, 7, 7, 1536)     │    10,783,535 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1536)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1536)           │         6,144 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       393,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,185,207 (42.67 MB)

 Trainable params: 398,600 (1.52 MB)

 Non-trainable params: 10,786,607 (41.15 MB)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        B3_CHECKPOINT_PATH,
        monitor="val_accuracy",
        save_best_only=True,
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=4,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.3,
        patience=2,
        verbose=1
    )
]


In [ ]:
history_b3_stage1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    class_weight=class_weights,
    callbacks=callbacks
)


Epoch 1/10
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.3373 - loss: 2.6109
Epoch 1: val_accuracy improved from -inf to 0.50287, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best_model_B3.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 156s 304ms/step - accuracy: 0.3374 - loss: 2.6098 - val_accuracy: 0.5029 - val_loss: 1.3646 - learning_rate: 0.0010
Epoch 2/10
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.4799 - loss: 1.4715
Epoch 2: val_accuracy did not improve from 0.50287
331/331 ━━━━━━━━━━━━━━━━━━━━ 25s 76ms/step - accuracy: 0.4799 - loss: 1.4715 - val_accuracy: 0.5007 - val_loss: 1.4355 - learning_rate: 0.0010
Epoch 3/10
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.5253 - loss: 1.1614
Epoch 3: val_accuracy improved from 0.50287 to 0.56595, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best_model_B3.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 32s 96ms/step - accuracy: 0.5253 - loss: 1.1613 - val_accuracy: 0.5659 - val_loss: 1.1831 - learning_rate: 0.0010
Epoch 4/10
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5443 - loss: 1.0832
Epoch 4: val_accuracy improved from 0.56595 to 0.61888, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best_model_B3.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 31s 93ms/step - accuracy: 0.5443 - loss: 1.0830 - val_accuracy: 0.6189 - val_loss: 1.0446 - learning_rate: 0.0010
Epoch 5/10
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.6021 - loss: 0.9398
Epoch 5: val_accuracy did not improve from 0.61888
331/331 ━━━━━━━━━━━━━━━━━━━━ 25s 77ms/step - accuracy: 0.6021 - loss: 0.9396 - val_accuracy: 0.5236 - val_loss: 1.2931 - learning_rate: 0.0010
Epoch 6/10
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.5865 - loss: 0.8568
Epoch 6: val_accuracy improved from 0.61888 to 0.62903, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best_model_B3.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 29s 87ms/step - accuracy: 0.5866 - loss: 0.8568 - val_accuracy: 0.6290 - val_loss: 1.0232 - learning_rate: 0.0010
Epoch 7/10
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.6154 - loss: 0.8746
Epoch 7: val_accuracy did not improve from 0.62903
331/331 ━━━━━━━━━━━━━━━━━━━━ 25s 77ms/step - accuracy: 0.6153 - loss: 0.8746 - val_accuracy: 0.6171 - val_loss: 1.0336 - learning_rate: 0.0010
Epoch 8/10
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.6131 - loss: 0.8039
Epoch 8: val_accuracy improved from 0.62903 to 0.63961, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best_model_B3.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 34s 102ms/step - accuracy: 0.6130 - loss: 0.8040 - val_accuracy: 0.6396 - val_loss: 0.9793 - learning_rate: 0.0010
Epoch 9/10
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.6259 - loss: 0.8067
Epoch 9: val_accuracy improved from 0.63961 to 0.64446, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best_model_B3.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 31s 93ms/step - accuracy: 0.6259 - loss: 0.8066 - val_accuracy: 0.6445 - val_loss: 0.9853 - learning_rate: 0.0010
Epoch 10/10
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.6369 - loss: 0.7874
Epoch 10: val_accuracy improved from 0.64446 to 0.64667, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/checkpoints/best_model_B3.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 34s 103ms/step - accuracy: 0.6368 - loss: 0.7874 - val_accuracy: 0.6467 - val_loss: 0.9681 - learning_rate: 0.0010


In [ ]:
base_model.trainable = True

fine_tune_at = int(len(base_model.layers) * 0.7)

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

print("Trainable layers:",
      len([l for l in base_model.layers if l.trainable]))


Trainable layers: 116


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
import os

DATA_DIR = "/content/drive/MyDrive/SkinDiseaseProject/clean_data"

print("Exists:", os.path.exists(DATA_DIR))
print("Folders:", os.listdir(DATA_DIR))


Exists: True
Folders: ['test', 'train', 'val']


In [ ]:
import tensorflow as tf

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42

DATA_DIR = "/content/drive/MyDrive/SkinDiseaseProject/clean_data"

train_ds_raw = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR + "/train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED,
    label_mode="int"
)

val_ds_raw = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR + "/val",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False,
    label_mode="int"
)


# ✅ SAVE CLASS NAMES HERE
CLASS_NAMES = train_ds_raw.class_names
print("Class names:", CLASS_NAMES)


Found 10580 files belonging to 8 classes.
Found 2267 files belonging to 8 classes.
Class names: ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'scc', 'vasc']


In [ ]:
CLASS_NAMES = train_ds_raw.class_names
print(CLASS_NAMES)


['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'scc', 'vasc']


In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input

def preprocess(image, label):
    image = preprocess_input(image)
    return image, label

train_ds = train_ds_raw.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_ds   = val_ds_raw.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds   = val_ds.prefetch(tf.data.AUTOTUNE)


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="sparse_categorical_crossentropy",   # 🔑 FIX
    metrics=["accuracy"]
)


In [ ]:
model.evaluate(val_ds, verbose=2)


71/71 - 52s - 726ms/step - accuracy: 0.6798 - loss: 0.9478


[0.9478135704994202, 0.6797530055046082]

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        "/content/drive/MyDrive/SkinDiseaseProject/models/final_model.keras",
        monitor="val_accuracy",
        save_best_only=True,
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=3,
        restore_best_weights=True
    )
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=12,
    callbacks=callbacks
)


Epoch 1/12
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step - accuracy: 0.7236 - loss: 0.7691
Epoch 1: val_accuracy improved from -inf to 0.73225, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/final_model.keras
331/331 ━━━━━━━━━━━━━━━━━━━━ 219s 534ms/step - accuracy: 0.7236 - loss: 0.7690 - val_accuracy: 0.7322 - val_loss: 0.7436
Epoch 2/12
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step - accuracy: 0.7321 - loss: 0.7250
Epoch 2: val_accuracy improved from 0.73225 to 0.74019, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/final_model.keras
331/331 ━━━━━━━━━━━━━━━━━━━━ 137s 413ms/step - accuracy: 0.7321 - loss: 0.7250 - val_accuracy: 0.7402 - val_loss: 0.7301
Epoch 3/12
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step - accuracy: 0.7464 - loss: 0.6940
Epoch 3: val_accuracy improved from 0.74019 to 0.74592, saving model to /content/drive/MyDrive/SkinDiseaseProject/models/final_model.keras
331/331 ━━━━━━━━━━━━━━━━━━━━ 136s 412ms/step - accuracy: 0.7464 - loss: 0.6940 -

In [ ]:
model = tf.keras.models.load_model(
    "/content/drive/MyDrive/SkinDiseaseProject/models/final_model.keras"
)

model.evaluate(val_ds, verbose=2)


71/71 - 37s - 515ms/step - accuracy: 0.7124 - loss: 0.9129


[0.9129153490066528, 0.7123952507972717]

In [ ]:
model.save(
    "/content/drive/MyDrive/SkinDiseaseProject/models/skin_disease_final.keras"
)


In [ ]:
from google.colab import files

files.download(
    "/content/drive/MyDrive/SkinDiseaseProject/models/final_model.keras"
)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>